<a href="https://colab.research.google.com/github/SattuSupCodes/SentimentAnalysis_TheoreticalLearning/blob/main/speaker_diarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import librosa
import numpy as np




In [3]:
!pip install --upgrade numpy scipy librosa

  Using cached numpy-2.4.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)


In [9]:
!pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=fafd9547b4a5717feea0607871e6b783ad521bfdf3805aa0460a77d72ed843dc
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [10]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("sample.wav")

segments = result["segments"]
for seg in segments:
    print(f"{seg['start']:.2f}-{seg['end']:.2f}: {seg['text']}")


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 101MiB/s]
/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


0.00-5.52:  Just that was kind of the thought and whatever and then you hit me and I was like no frickin way
5.52-9.40:  He's hitting me right now. So I was like absolutely. This is like perfect
9.40-15.50:  It's the universe just putting it in perfect timing because that makes me so happy because it was exactly the opposite way
15.50-21.76:  So I've for a long time had people that I'd love to sit down with and every time I saw you do something with mental health and
21.76-26.54:  Vogue or any of the segments that you ever did I was just like she's amazing like you know
26.54-29.66:  It's incredible to the way you were talking about it when you'd post about it
29.66-33.82:  I'd be like oh this is incredible like this is see you speaking so openly about it
33.82-39.74:  Invulnerably about it and so I've always wanted to do this and then but we've never really crossed paths
39.74-44.54:  And so I'm always just a bit like and I don't like asking friends of friends when I haven't met someo

In [4]:
y, sr = librosa.load("sample.wav", sr=None)


In [13]:
segment_len = int(1.5 * sr)
segments = [
    y[i:i+segment_len]
    for i in range(0, len(y), segment_len)
    if len(y[i:i+segment_len]) == segment_len
]


In [15]:


y, sr = librosa.load("sample.wav", sr=None)

segment_len_sec = 1.5
segment_len = int(segment_len_sec * sr)

segments_audio = [
    y[i:i+segment_len]
    for i in range(0, len(y), segment_len)
    if len(y[i:i+segment_len]) == segment_len
]


In [14]:
def extract_mfcc(seg, sr):
    mfcc = librosa.feature.mfcc(y=seg, sr=sr, n_mfcc=13)
    return np.mean(mfcc, axis=1)

X = np.array([extract_mfcc(seg, sr) for seg in segments])


In [16]:
def mfcc_embed(seg, sr):
    mfcc = librosa.feature.mfcc(y=seg, sr=sr, n_mfcc=13)
    return np.mean(mfcc, axis=1)

X = np.array([mfcc_embed(seg, sr) for seg in segments_audio])


# Agglomerative Clustering
basically, we segmented our audio into many segments of 1.5 secs (we assume that thee spealers are consistent)

then our already-existng (thank you python-gods) agglomerative clustering helps in "this segment sounds very similar to this segment. CLUSTER" and boom

speaker 0 and speaker 1

In [7]:
from sklearn.cluster import AgglomerativeClustering

clusterer = AgglomerativeClustering(n_clusters=2)
labels = clusterer.fit_predict(X)


In [17]:
from sklearn.cluster import AgglomerativeClustering

labels = AgglomerativeClustering(n_clusters=2).fit_predict(X)
from sklearn.cluster import AgglomerativeClustering

labels = AgglomerativeClustering(n_clusters=2).fit_predict(X)


In [19]:
for i, label in enumerate(labels):
    start = i * 1.5
    end = start + 1.5
    print(f"{start:.1f}s - {end:.1f}s : Speaker {label}")


0.0s - 1.5s : Speaker 0
1.5s - 3.0s : Speaker 1
3.0s - 4.5s : Speaker 1
4.5s - 6.0s : Speaker 1
6.0s - 7.5s : Speaker 1
7.5s - 9.0s : Speaker 1
9.0s - 10.5s : Speaker 1
10.5s - 12.0s : Speaker 1
12.0s - 13.5s : Speaker 0
13.5s - 15.0s : Speaker 0
15.0s - 16.5s : Speaker 0
16.5s - 18.0s : Speaker 0
18.0s - 19.5s : Speaker 0
19.5s - 21.0s : Speaker 0
21.0s - 22.5s : Speaker 0
22.5s - 24.0s : Speaker 0
24.0s - 25.5s : Speaker 0
25.5s - 27.0s : Speaker 0
27.0s - 28.5s : Speaker 0
28.5s - 30.0s : Speaker 0
30.0s - 31.5s : Speaker 0
31.5s - 33.0s : Speaker 0
33.0s - 34.5s : Speaker 0
34.5s - 36.0s : Speaker 0
36.0s - 37.5s : Speaker 1
37.5s - 39.0s : Speaker 0
39.0s - 40.5s : Speaker 1
40.5s - 42.0s : Speaker 0
42.0s - 43.5s : Speaker 1
43.5s - 45.0s : Speaker 0
45.0s - 46.5s : Speaker 1
46.5s - 48.0s : Speaker 1
48.0s - 49.5s : Speaker 1
49.5s - 51.0s : Speaker 1
51.0s - 52.5s : Speaker 1
52.5s - 54.0s : Speaker 1
54.0s - 55.5s : Speaker 1


In [22]:
labels = labels.flatten()

def assign_speaker(start_time):
    idx = int(start_time / segment_len_sec)
    if idx < 0 or idx >= len(labels):
        return -1
    return int(labels[idx])

for seg in result["segments"]:
    spk = assign_speaker(seg["start"])
    print(f"[Speaker {spk}] {seg['text']}")

[Speaker 0]  Just that was kind of the thought and whatever and then you hit me and I was like no frickin way
[Speaker 1]  He's hitting me right now. So I was like absolutely. This is like perfect
[Speaker 1]  It's the universe just putting it in perfect timing because that makes me so happy because it was exactly the opposite way
[Speaker 0]  So I've for a long time had people that I'd love to sit down with and every time I saw you do something with mental health and
[Speaker 0]  Vogue or any of the segments that you ever did I was just like she's amazing like you know
[Speaker 0]  It's incredible to the way you were talking about it when you'd post about it
[Speaker 0]  I'd be like oh this is incredible like this is see you speaking so openly about it
[Speaker 0]  Invulnerably about it and so I've always wanted to do this and then but we've never really crossed paths
[Speaker 1]  And so I'm always just a bit like and I don't like asking friends of friends when I haven't met someone a